# Viewing angles

Visualizing the illumination and viewing angles. Explaining how ViSiProg groups are related?

In [47]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import numpy as np
import os

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import plotly.graph_objs as go
from visiprog.data import read_VSP_label, read_img_list, read_viewing_conditions, viewing_condition_index
from visiprog.data import read_viewing_conditions, polar_to_euclidean, angle_between
from visiprog.data import illum_spatial_adjacent_graph, viewing_spatial_adjacent_graph
from IPython.display import display, HTML

from scipy.spatial import Voronoi, SphericalVoronoi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
df_viewing = read_viewing_conditions()

np.savetxt('df_viewing.csv', df_viewing, fmt='%.4f', delimiter=',')

print(df_viewing.columns.values)
print(df_viewing.shape)

['view_theta' 'view_phi' 'illum_theta' 'illum_phi' 'illum_x' 'illum_y'
 'illum_z' 'view_x' 'view_y' 'view_z']
(205, 10)


In [25]:
def make_image_html(image_path):
     return '<img src="{}" width="100" height="100" style="display:inline;margin:1px"/>'.format(image_path)

In [36]:
groups = read_VSP_label()
list_img = read_img_list()

traces = []

group = groups[169]

for i in group: 
    viewing_index = viewing_condition_index(list_img[i])
    
    illum_x = df_viewing['illum_x'][viewing_index]
    illum_y = df_viewing['illum_y'][viewing_index]
    illum_z = df_viewing['illum_z'][viewing_index]

    view_x = df_viewing['view_x'][viewing_index]
    view_y = df_viewing['view_y'][viewing_index]
    view_z = df_viewing['view_z'][viewing_index]
    
    line_x = [illum_x, illum_x, 0, view_x]
    line_y = [illum_y - 0.05, illum_y, 0, view_y]
    line_z = [illum_z,illum_z, 0, view_z]
    
    # plotting line
    line_scatter = go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z,
        name=os.path.basename(list_img[i])
    )
    
    traces.append(line_scatter)

layout = go.Layout(
    title='Angles of VSP group',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='vspgroup')

# display the images as well
display(HTML(''.join(make_image_html(list_img[i]) for i in group) + '<br>' + \
            '\t'.join(os.path.basename(list_img[i]) for i in group)))

# Formal analysis using graph and minimum spanning tree

In [33]:
groups = read_VSP_label(pappas_only=True, sorted_by_material=True)
list_img = read_img_list()

print(len(groups))

mst_distances = []

for group in groups:
    
    # building the weighted undirected graph
    G = nx.Graph()
    G.add_nodes_from(range(len(group)))

    for i in range(len(group)):
        index_i = viewing_condition_index(list_img[group[i]])
        
        illum_x_1 = df_viewing['illum_x'][index_i]
        illum_y_1 = df_viewing['illum_y'][index_i]
        illum_z_1 = df_viewing['illum_z'][index_i]
        
        view_x_1 = df_viewing['view_x'][index_i]
        view_y_1 = df_viewing['view_y'][index_i]
        view_z_1 = df_viewing['view_z'][index_i]
        
        for j in range(i + 1, len(group)):
            index_j = viewing_condition_index(list_img[group[j]])
            illum_x_2, illum_y_2, illum_z_2 = polar_to_euclidean(df_viewing['illum_theta'][index_j],df_viewing['illum_phi'][index_j])
            view_x_2, view_y_2, view_z_2 = polar_to_euclidean(df_viewing['view_theta'][index_j],df_viewing['view_phi'][index_j])
            
            weight_illum = angle_between((illum_x_1, illum_y_1, illum_z_1),(illum_x_2, illum_y_2, illum_z_2))
            weight_view = angle_between((view_x_1, view_y_1, view_z_1),(view_x_2, view_y_2, view_z_2))
        
            G.add_edge(i, j, weight=weight_illum)
            G.add_edge(i, j, weight=weight_view)
            
#     print(G.edges(data=True))
#     print("\n\nSorting the MST\n\n")
    
    # minimum spanning tree
    mst=nx.minimum_spanning_edges(G,data=True)
    
    distances = []
    for e in mst:
        distances.append(e[2]['weight'])
        
    mst_distances.append(distances)

456


In [20]:
mst_out = np.asarray(mst_distances)
np.savetxt('mst_distances.csv', mst_out, fmt='%.4f', delimiter=',')
print(mst_out.shape)

(456, 8)


(604, 8)


# Formal analysis using graph and VORONOI DIAGRAM

Why? Since angle does not explain a lot. It could be better explained through spatial adjacency.

In [48]:
# building spatial graph for illumination angles
G_illum = illum_spatial_adjacent_graph()
G_viewing = viewing_spatial_adjacent_graph()

In [69]:
nx.shortest_path_length(G_illum,source = 0, target=2)

1

In [58]:
groups, _ = read_VSP_label(pappas_only=True, sorted_by_material=True)
list_img = read_img_list()

mst_spatial_distances = []

for group in groups:    

    # building the weighted undirected graph
    G = nx.Graph()
    G.add_nodes_from(range(len(group)))

    for i in range(len(group)):
        m_i = viewing_condition_index(list_img[group[i]])
        for j in range(i + 1, len(group)):
            m_j = viewing_condition_index(list_img[group[j]])
            weight_illum = nx.shortest_path_length(G_illum, source=m_i - 1, target=m_j - 1)
            weight_view = nx.shortest_path_length(G_viewing, source=m_i - 1, target=m_j - 1)                
            G.add_edge(i, j, weight = min(weight_illum, weight_view))
            
    mst=nx.minimum_spanning_edges(G,data=True)
    
    distances = []   
    for e in mst:
        distances.append(e[2]['weight'])
        
    mst_spatial_distances.append(distances)

In [59]:
np.savetxt('mst_spatial_distances.csv', np.array(mst_spatial_distances), fmt='%d', delimiter=',')

In [60]:
print(np.array(mst_spatial_distances))

[[1 1 1 ..., 3 3 6]
 [1 1 1 ..., 3 4 5]
 [0 1 1 ..., 1 1 2]
 ..., 
 [1 1 1 ..., 1 2 3]
 [1 1 1 ..., 1 2 2]
 [1 1 1 ..., 2 2 4]]
